In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#!ls "/content/gdrive/MyDrive/"

In [3]:
!pip install foolbox

     |████████████████████████████████| 1.7MB 8.0MB/s 
     |████████████████████████████████| 174kB 54.3MB/s 
     |████████████████████████████████| 61kB 8.7MB/s 
     |████████████████████████████████| 71kB 11.5MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np
import foolbox as fb
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import LinfPGD,LinfBasicIterativeAttack,LinfFastGradientAttack,L2CarliniWagnerAttack,LinfDeepFoolAttack,L2DeepFoolAttack,L2BasicIterativeAttack,L2ProjectedGradientDescentAttack

class Model_Drop(nn.Module):
    def __init__(self):
        super(Model_Drop, self).__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=3,padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3,padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3,padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3,padding=1)
        self.fc1 = nn.Linear(7*7*64, 600)
        self.fc2 = nn.Linear(600, 128)
        self.fc3 = nn.Linear(128, 10)
        self.drop_layer = nn.Dropout(p=0.25)

    def last_hidden_layer_output(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = self.drop_layer(F.relu(self.conv3(x)))
        x = self.drop_layer(F.relu(self.conv4(x)))
        x = x.view(-1, 7*7*64)
        x = self.drop_layer(F.relu(self.fc1(x)))
        x = F.relu(self.fc2(x))
        return x

    def forward(self, x):
        x = self.last_hidden_layer_output(x)
        x = self.fc3(x)
        return x

train_data = datasets.FashionMNIST(root='data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root='data', train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.shape[0], -1)

torch.manual_seed(2)
torch.cuda.manual_seed(2)

softmax = nn.Softmax(dim=1)

def enable_dropout(model):
    """ Function to enable the dropout layers during test-time """
    for m in model.modules():
        if m.__class__.__name__.startswith('Dropout'):
            m.train()

criterion = nn.CrossEntropyLoss()

T = 100

def epoch(loader, model, opt=None):

    if opt:
        model.train()
    else:
        model.eval()

    total_loss, total_err = 0., 0.

    for X, y in loader:
        X, y = X.to(device), y.to(device)
        
        outputs = model(X)

        log_prob_s = F.log_softmax(outputs / T, dim=1)

        loss = F.nll_loss(log_prob_s,y)
        #loss = criterion(log_prob_s, y)

        if opt:
            opt.zero_grad()
            loss.backward()
            opt.step()

        total_err += (outputs.max(dim=1)[1] != y).sum().item()
        total_loss += loss.item() * X.shape[0]
    return total_err / len(loader.dataset), total_loss / len(loader.dataset)




Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
model_cnn = Model_Drop()
model_cnn.to(device)

opt = optim.Adam(model_cnn.parameters(), lr=0.001)
#opt = optim.SGD(model_cnn.parameters(), lr=0.1, momentum=0.5)

for te in range(50):

    #if te == 4:
      #for param_group in opt.param_groups:
        #param_group["lr"] = 1e-3

    train_err, train_loss = epoch(train_loader, model_cnn, opt)
    test_err, test_loss = epoch(test_loader, model_cnn) 

    print(*("{:.6f}".format(i) for i in (train_err, test_err)), sep="\t")
    #print(train_err)



/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


0.303967	0.216800
0.180533	0.156900
0.148117	0.140700
0.131317	0.128900
0.118733	0.110500
0.111717	0.106900
0.102117	0.105200
0.097283	0.099000
0.092217	0.098900
0.088633	0.096500
0.083700	0.087800
0.080167	0.091100
0.076017	0.086200
0.072833	0.084100
0.070900	0.084000
0.067850	0.083100
0.064950	0.078200
0.061317	0.076900
0.060017	0.079300
0.056733	0.079100
0.055117	0.077900
0.052450	0.081400
0.051233	0.076600
0.047617	0.075500
0.045567	0.076100
0.043267	0.076300
0.042950	0.075100
0.040533	0.076500
0.039533	0.077000
0.036400	0.075600
0.035783	0.074400
0.034367	0.073200
0.032733	0.074600
0.032617	0.075100
0.029717	0.073300
0.029633	0.073400
0.028433	0.075400
0.028317	0.076800
0.025150	0.075900
0.027100	0.076900
0.023583	0.075800
0.023283	0.074900
0.023033	0.075300
0.022100	0.077500
0.022150	0.075700
0.020217	0.076300
0.020667	0.075200
0.018933	0.075800
0.018883	0.072300
0.018800	0.076900


In [6]:
model_cnn.eval()
torch.save(model_cnn.state_dict(), "TENET_FASHION_MNIST_model_cnn_teacher_T_100.pt")

model_cnn.load_state_dict(torch.load("TENET_FASHION_MNIST_model_cnn_teacher_T_100.pt"))
model_cnn.eval()

Model_Drop(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=3136, out_features=600, bias=True)
  (fc2): Linear(in_features=600, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
  (drop_layer): Dropout(p=0.25, inplace=False)
)

In [7]:
torch.save(model_cnn.state_dict(), "/content/gdrive/MyDrive/TENET_FASHION_MNIST_model_cnn_teacher_T_100.pt")